# Sparse Word Representations Assignment (50 points)

**Objective:** Implement and apply sparse word representations using co-occurrence counts and TF-IDF weighting on a small document collection.

**Dataset:** We will use the 20 Newsgroups dataset, readily available in scikit-learn. We'll limit it to a subset of categories and a reduced number of documents to keep things manageable.

```
Instructions:
Fill out the parts that are specified with
% -- Your Implementation -- %
```

### 1. Data Loading and Preprocessing (5 points)

Steps:
- Load the 20 Newsgroups dataset using scikit-learn, but only select 4 categories of your choice.
- Limit the dataset to the first 1000 documents.
- Tokenize the documents into words. You can use a simple tokenizer like splitting on spaces and remove any punctuations.
- Create a vocabulary of the 5000 most frequent words across all documents.

In [1]:
#
!pip install scikit-learn
from typing import List, Set, Dict, Tuple, Union
from collections import Counter
import numpy as np
from math import log
from sklearn.datasets import fetch_20newsgroups

In [7]:
from sklearn.datasets import fetch_20newsgroups

# Load the dataset with selected categories
categories = ['talk.politics.misc', 'sci.med']
newsgroups_data = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

In [9]:
import re
from collections import Counter
from typing import List, Set

def preprocess_text(text: str) -> List[str]:
    """
    Tokenize and clean text by:
    1. Converting to lowercase
    2. Removing punctuation
    3. Splitting on whitespace
    4. Removing empty tokens
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    tokens = [token for token in tokens if token]
    return tokens

# Get first 1000 documents
documents = newsgroups_data.data[:2000]

# Tokenize all documents
tokenized_docs = [preprocess_text(doc) for doc in documents]

# Now write a fucntion to get the vocabulary
def get_vocab(tokenized_docs: List[List[str]], vocab_size: int) -> Set[str]:
    """
    Create vocabulary from tokenized documents by:
    1. Counting word frequencies across all documents
    2. Taking the `vocab_size` most common words

    Args:
        tokenized_docs: List of documents, where each document is a list of tokens

    Returns:
        Set of vocabulary words
    """
    all_tokens = []
    for doc in tokenized_docs:
        all_tokens.extend(doc)
    counter = Counter(all_tokens)
    most_common_words = counter.most_common(vocab_size)
    vocab = set([word for word, _ in most_common_words])
    return vocab

vocab = get_vocab(tokenized_docs, vocab_size=5000)

print(f"Vocabulary size: {len(vocab)}")
print("\nSample vocabulary words:")
print(list(vocab)[:10])


Vocabulary size: 5000

Sample vocabulary words:
['protecting', 'classes', 'earth', 'ice', 'distilled', '52', 'spikes', 'not', 'property', 'applied']


## 2. Computing the co-occurrence matrix (10 points)

Assume co-occurrence means if a word appears in the same document and within a window of size 10 of another word. Compute the co-occurrence matrix based on raw frequencies.

For example in the sentence: "the hungry cat ate the small fish near the blue lake"

With window size 6, the co-occurrences for the word "cat":

Left context (within 2 words): "the", "hungry"
Right context (within 3 words): "ate", "the", "small"
So "cat" co-occurs once each with: hungry, ate, small
And it co-occurs twice with the word: "the"

Words outside the window like "fish", "near", "blue", "lake" are not counted as co-occurrences with "cat" in this case.

Complete the following steps:

- Construct a co-occurrence matrix. This matrix will have dimensions `vocab_size` x `vocab_size` (5000 x 5000 in our case).
- For each document, iterate through its words. For each word, increment the corresponding entries in the co-occurrence matrix for its neighboring words within a specified window size (e.g., a window of 5 words to the left and right).


In [11]:
import numpy as np

def compute_cooccurrence_matrix(tokenized_docs: List[List[str]], vocab: Set[str], window_size: int) -> np.ndarray:
    """
    Compute word co-occurrence matrix from tokenized documents using a sliding window approach.

    Args:
        tokenized_docs (List[List[str]]): List of documents where each document is a list of tokens
        vocab (Set[str]): Set of vocabulary words to consider
        window_size (int): Number of words to consider
            if odd, there are window_size//2 words on each side of the center word
            if even, there are window_size//2 words on the left and window_size//2 - 1 words on the right

    Returns:
        np.ndarray: Co-occurrence matrix of shape (vocab_size, vocab_size) where entry [i,j]
                   represents how many times word i appears within the window of word j
    """
    vocab_list = sorted(list(vocab))
    vocab_size = len(vocab_list)
    word2idx = {word: i for i, word in enumerate(vocab_list)}
    cooc_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int64)
    for doc in tokenized_docs:
      doc_length = len(doc)
      for i, word in enumerate(doc):
        if word not in vocab:
          continue
        word_idx = word2idx[word]
        left_context = max(0, i - window_size//2)
        right_context = min(doc_length, i + window_size//2 + 1)
        for j in range(left_context, right_context):
          if j == i:
            continue
          context_word = doc[j]
          if context_word in vocab:
            context_idx = word2idx[context_word]
            cooc_matrix[word_idx, context_idx] += 1
    return cooc_matrix


cooccurrence_matrix = compute_cooccurrence_matrix(tokenized_docs, vocab, window_size=10)
print(f"Co-occurrence matrix shape: {cooccurrence_matrix.shape}")

Co-occurrence matrix shape: (5000, 5000)

Sample co-occurrence matrix:
[[0 0 0 1 1]
 [0 4 1 0 0]
 [0 1 0 0 0]
 [1 0 0 8 3]
 [1 0 0 3 2]]


## 3. Application: Word Similarity (10 points)

- Implement a function to calculate the cosine similarity between two *word* vectors.
- For the following words, find the 5 most similar words to each of your chosen words based on their raw co-occurrence representations using the cosine similarity function.
[concerned, lone, matthew]
- Print the chosen words and their 5 most similar words, along with their similarity scores.

Recall that `Cosine Similarity(A, B) = (A . B) / (||A|| * ||B||)`
where `.` denotes the dot product, and `||A||` is the magnitude (Euclidean norm) of vector A.


In [24]:
words = ['concerned', 'lone', 'matthew']

def cosine_similarity(vec1: np.ndarray, vec2: np.ndarray) -> float:
    """Calculate cosine similarity between two vectors."""
    if np.linalg.norm(vec1) == 0 or np.linalg.norm(vec2) == 0:
        return 0
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))


def find_most_similar_words(words: List[str], cooccurrence_matrix: np.ndarray) -> List[Tuple[str, List[Tuple[str, float]]]]:
    """
    Find the 5 most similar words to each input word based on their co-occurrence representations.

    Args:
        words (List[str]): List of words to find similar words for
        cooccurrence_matrix (np.ndarray)

    Returns:
        List[Tuple[str, List[Tuple[str, float]]]]: For each input word, returns a tuple containing:
            - The input word
            - A list of 5 tuples, each containing:
                - A similar word
                - The cosine similarity score between the input and similar word

    Example:
        >>> result = find_most_similar_words(['cat'], cooccurrence_matrix)
        >>> result
        [('cat', [('dog', 0.85), ('kitten', 0.76), ('pet', 0.72), ...])]
    """
    word2idx = {word: i for i, word in enumerate(vocab)}
    idx2word = {i: word for word, i in word2idx.items()}
    result = []
    for word in words:
        if word not in vocab:
            continue
        word_idx = word2idx[word]
        word_vector = cooccurrence_matrix[word_idx]
        similarities = [(idx2word[i], cosine_similarity(word_vector, cooccurrence_matrix[i])) for i in range(len(vocab)) if i != word_idx]
        similarities.sort(key=lambda x: x[1], reverse=True)
        result.append((word, similarities[:5]))
    return result


print(find_most_similar_words(words, cooccurrence_matrix))

[('concerned', [('animals', 0.6514739218413861), ('europe', 0.628410287203555), ('added', 0.6119238072666792), ('favor', 0.6091273377913761), ('36', 0.6072338280733376)]), ('lone', [('credibility', 0.8162868810391121), ('sitting', 0.8124927632356572), ('debunking', 0.8120074707937157), ('epidemic', 0.8091645459422537), ('meal', 0.8085538590593634)]), ('matthew', [('relation', 0.8326262912298619), ('slow', 0.8324409724104836), ('located', 0.8282410234341334), ('grants', 0.8272675439506562), ('formerly', 0.8237826324581541)])]


## 4. TF-IDF weighting (10 points)

In this part you will implement TF-IDF weighting, which is a numerical statistic that reflects how important a word is to a document in a collection or corpus. TF-IDF consists of two components:
1. Term Frequency (TF): How frequently a word appears in a document  
2. Inverse Document Frequency (IDF): How unique or rare the word is across all documents  

Assume we have a term-document matrix of co-occurrences.
Then we can use TF-IDF to reweight the raw frequences.

First compute the TF. Recall that TF is:

tf(t,d) = 1 + log(count(t,d)) if term t appears in document d  
tf(t,d) = 0 if term t doesn't appear in document d

In [25]:
def compute_tf(document: Union[str, List[str]], vocab: Set[str]) -> Dict[str, float]:
    """
    Compute term frequencies in a document.

    """
    counter = Counter(document)
    tf_dict = {}
    for word in vocab:
        if word in counter:
            tf_dict[word] = 1 + log(counter[word])
        else:
            tf_dict[word] = 0
    return tf_dict

### Compute inverse document frequency
Implement a function to calculate document frequencies across a corpus.
Recall that inverse document frequency is:

log ( N / df ), where:

- **N** is the total number of documents in the corpus.
- **df** is the document frequency, i.e., the number of documents in which the term appears.

In [26]:
def compute_idf(documents: Union[List[str], List[List[str]]], vocab: Set[str]) -> Dict[str, float]:
    """
    Compute inverse document frequency for all terms in the corpus.
    Returns a dictionary mapping terms to their IDF scores.
    """
    N = len(documents)
    df = {}
    for word in vocab:
        df[word] = sum(1 for doc in documents if word in doc)
    word2idf = {word: log(N / (1 + df[word])) for word in vocab}
    return word2idf

### Caclculate the co-occurrence matrix with TF-IDF weights.

Now we want to calculate TF-IDF scores for each term in each document. We'll combine the term frequency (TF) and inverse document frequency (IDF) scores we computed earlier to get the final TF-IDF weights.  
This will give us a measure of how important each word is to a document in the context of the entire corpus.

Recall that the TF-IDF score is calculated as:  
`tf-idf = tf * idf`  

Where:  
- tf is the term frequency (number of times term appears in document)  
- idf is the inverse document frequency `(log(N/df))`


In [28]:
def compute_tfidf(tokenized_docs: List[List[str]], vocab: Set[str]) -> np.ndarray:
    """
    Compute TF-IDF (Term Frequency-Inverse Document Frequency) matrix for a collection of documents.

    The TF-IDF score is calculated as:
    tf-idf(t,d) = tf(t,d) * idf(t)

    Args:
        tokenized_docs (List[List[str]]): List of documents, where each document is a list of tokens
        vocab (Set[str]): Set of vocabulary words to consider

    Returns:
        np.ndarray: TF-IDF matrix of shape (vocab_size, num_documents) where:
            - Each row represents a term from the vocabulary
            - Each column represents a document
            - Entry [i,j] represents the TF-IDF score of term i in document j

    Note:
        - IDF calculation includes +1 smoothing to avoid division by zero
        - TF calculation uses log normalization: 1 + log(term_count)
    """
    word2idf = compute_idf(tokenized_docs, vocab)
    tfidf_matrix = np.zeros((len(vocab), len(tokenized_docs)))
    for i, doc in enumerate(tokenized_docs):
        tf_dict = compute_tf(doc, vocab)
        for j, word in enumerate(vocab):
            tfidf_matrix[j, i] = tf_dict[word] * word2idf[word]
    return tfidf_matrix

# Compute TF-IDF matrix
word_document_matrix = compute_tfidf(tokenized_docs, vocab)

### Application in search (10 points)

Now we will use TF-IDF representations to implement a simple search function.
In this function we match the most similar documents to a given query.
The query itself should be converted to a TF-IDF vector as above.

Implement the following function.

In [31]:
def search_documents(query: str, tokenized_docs: List[List[str]], tfidf_matrix: np.ndarray, vocab: Set[str]) -> List[Tuple[int, float]]:
    """
    Search for documents most relevant to a query using TF-IDF representations.

    Args:
        query (str): Search query string
        tokenized_docs (List[List[str]]): List of documents where each document is a list of tokens
        tfidf_matrix (np.ndarray): TF-IDF matrix of shape (vocab_size, num_documents)
        vocab (Set[str]): Vocabulary set used for TF-IDF computation

    Returns:
        List[Tuple[int, float]]: List of (document_index, similarity_score) tuples,
            sorted by relevance (highest similarity first)
    """
    vocab_list = sorted(vocab)
    word_to_index = {word: idx for idx, word in enumerate(vocab_list)}
    query_tokens = query.lower().split()

    N = len(tokenized_docs)
    df_counts = {word: 0 for word in vocab_list}
    for doc in tokenized_docs:
        unique_tokens = set(token.lower() for token in doc)
        for token in unique_tokens:
            if token in df_counts:
                df_counts[token] += 1
    idf = {word: np.log((N + 1) / (df_counts[word] + 1)) + 1 for word in vocab_list}
    query_tf = {}
    for token in query_tokens:
        if token in word_to_index:
            query_tf[token] = query_tf.get(token, 0) + 1
    query_vec = np.zeros(len(vocab_list))
    for token, count in query_tf.items():
        idx = word_to_index[token]
        query_vec[idx] = count * idf[token]
    dot_products = np.dot(query_vec, tfidf_matrix)
    query_norm = np.linalg.norm(query_vec)
    doc_norms = np.linalg.norm(tfidf_matrix, axis=0)
    if query_norm == 0:
        similarities = np.zeros(tfidf_matrix.shape[1])
    else:
        similarities = dot_products / (query_norm * doc_norms)
        similarities = np.nan_to_num(similarities)
    results = list(enumerate(similarities))
    results.sort(key=lambda x: x[1], reverse=True)

    return results



# Example usage:
query = "medical treatment for heart disease"
results = search_documents(query, tokenized_docs, word_document_matrix, vocab)

print(f"Top 5 most relevant documents for query: '{query}'\n")
for doc_idx, score in results[:5]:
    # Print first 100 characters of each document
    preview = ' '.join(tokenized_docs[doc_idx])[:100] + "..."
    print(f"Document {doc_idx} (score: {score:.3f}):")
    print(f"{preview}\n")

Top 5 most relevant documents for query: 'medical treatment for heart disease'

Document 225 (score: 0.217):
im curious why you think that particular adjective is important...

Document 32 (score: 0.172):
its not a cycle free helium will escape from the atmosphere due to its high velocity it wont be prac...

Document 391 (score: 0.147):
gordon banks this certainly describes my situation perfectly for me there is a constant dynamic betw...

Document 146 (score: 0.137):
what kind of brainless clod doesnt understand the difference between a proposed bill blocked in cong...

Document 694 (score: 0.137):
hey i wasnt the one dancing and singing on jan 20 now was i i was roundly ridiculed for my predictio...



<ipython-input-31-634e602ebb04>:54: RuntimeWarning: invalid value encountered in divide
  similarities = dot_products / (query_norm * doc_norms)


## 5. Reflection questions (10 points)

Now please answer the following reflection questions:

1- How do the most similar words differ when using raw co-occurrence counts versus TF-IDF weighted representations? What might explain these differences?

# Your answer here
    The similar word genearate by raw co-occurrence is affected by the high frequence word around that word. So the similar word might not have a strong association with the word. TF-IDF weighting adjusts these counts by reducing the impact of frequence, instead weighting more the importance in the whole document that carry more unique or context-specific information.
    Which makes the TF-IDF generating more related word.

2. What are the limitations of using a fixed window size for co-occurrence counting? How might this impact the quality of word similarities for different types of words (e.g., nouns vs. verbs)?

# Your answer here
    The first limitation is that the important words might be outside the window sized, and large windowsize might include too much useless information.
    Also different word might need different window sized to fetch the enough informaion. For example the verb and adj might have different effected window sized
    

3- How does the choice of preprocessing steps (like removing punctuation, converting to lowercase, etc.) affect the resulting word representations? Can you think of cases where preserving some of this information might be beneficial?

# Your answer here
    Preprocessing steps can effectively remove repeat words such as a same word in the begining of a sentence. This can help the model learn more meaningful information. However sometimes the punctuation can be useful for example ! can giving the same word different emotion thus influence the meaning of a sentence.